In [ ]:
from autogen import ConversableAgent, LLMConfig, run
from autogen.events.agent_events import InputRequestEvent
from autogen.io.event_processors import ConsoleEventProcessor
from autogen.tools import tool
from autogen.agentchat.termination import Keyword, MaxTurns

llm_config = LLMConfig.from_json(path="OAI_CONFIG_LIST").where(model=["gpt-4o-mini"], api_type="openai")

llm_config

# Single agent run

In [ ]:
@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


with llm_config:
    agent = ConversableAgent(name="Alice")

get_meaning_of_life.register_for_llm(agent)

response = run(
    agent,  # or receiver=agent
    message="What is the meaning of life?",  # or message="What is the meaning of life?"
    terminate_on=MaxTurns(4) | Keyword("TERMINATE"),  # implement arithmetic for this
    # default values for the following arguments:
    # sender="user",  # if not specified, we assume the sender is the user
    # run_pattern=RoundRobinRunPattern(),  # default run pattern is round robin
)


for event in response.events:
    print(f"Got event: {event}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.content.prompt))

print(response.summary)

## With console processor

In [ ]:
from autogen import UserProxyAgent


@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


with llm_config:
    agent = ConversableAgent(name="Alice")

user_proxy = UserProxyAgent("user_proxy")

get_meaning_of_life.register_for_llm(agent)

response = run(sender=user_proxy, receiver=agent, message="What is the meaning of life?")

processor = ConsoleEventProcessor()
processor.process(response)

print(response.summary)

# Two agent run

In [ ]:
with llm_config:
    agent_1 = ConversableAgent(name="Jane")
    agent_2 = ConversableAgent(name="Bob")

response = run(sender=agent_1, receiver=agent_2, message="have a joke off", terminate_on=MaxTurns(5))


for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

## With event processor

In [ ]:
with llm_config:
    agent_1 = ConversableAgent(name="Jane")
    agent_2 = ConversableAgent(name="Bob")

response = run(sender=agent_1, receiver=agent_2, message="have a joke off", terminate_on=Keyword("TERMINATE"))


ConsoleEventProcessor().process(response)

print(response.summary)

# Multi agent run with chat manager

In [ ]:
from autogen.run_patterns import GroupRunPattern


@tool()
def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


with llm_config:
    planner = ConversableAgent(
        name="Planner",
        system_message="You are a planner. Collaborate with teacher and reviewer to create lesson plans.",
    )

    reviewer = ConversableAgent(
        name="Reviewer",
        system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    )

    teacher = ConversableAgent(
        name="Teacher",
        system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    )
    run_pattern = GroupRunPattern(planner, reviewer, teacher)

submit_plan.register_for_llm(teacher)

response = run(
    receiver=planner,
    message="Create lesson plans for 4th grade.",
    run_pattern=run_pattern,
    terminate_on=Keyword("DONE!") | MaxTurns(10),
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

# Swarm chat manager

In [ ]:
from autogen import AfterWorkOption, ConversableAgent
from autogen.run_patterns import SwarmRunPattern

planner_message = """You are a classroom lesson planner.
Given a topic, write a lesson plan for a fourth grade class.
If you are given revision feedback, update your lesson plan and record it.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum
and provide a maximum of 3 recommended changes for each review.
Make sure you provide recommendations each time the plan is updated.
"""

teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
"""

with llm_config:
    # 1. Create our agents

    planner = ConversableAgent(name="planner_agent", system_message=planner_message)
    reviewer = ConversableAgent(name="reviewer_agent", system_message=reviewer_message)
    teacher = ConversableAgent(name="teacher_agent", system_message=teacher_message)

    run_pattern = SwarmRunPattern(after_work=AfterWorkOption.SWARM_MANAGER)

response = run(
    planner,
    message="Today, let's introduce our kids to the solar system.",
    run_pattern=run_pattern,
    terminate_on=MaxTurns(10),
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

# Sequential chat

Sequential chat can be implemented by calling the run sequentially

In [ ]:
with llm_config:
    planner = ConversableAgent(
        name="Planner",
        system_message="You are a planner. Create lesson plans.",
    )

    reviewer = ConversableAgent(
        name="Reviewer",
        system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    )

    teacher = ConversableAgent(
        name="Teacher",
        system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    )
    run_pattern = GroupRunPattern(planner, reviewer, teacher)


sequential_chats = [
    (teacher, planner, "Create lesson plans for 4th grade."),
    (planner, reviewer, "Review lesson plans against 4th grade curriculum."),
    (reviewer, teacher, "Review lesson plans against 4th grade curriculum."),
]

context = ""

for sender, receiver, message in sequential_chats:
    message = f"[{message}\n\n context: {context}]"
    response = run(
        sender=sender,
        receiver=receiver,
        message=message,
        run_pattern=run_pattern,
        terminate_on=Keyword("DONE!") | MaxTurns(10),
    )

    for event in response.events:
        print(f"Got event: {event.type=}, {event=}")
        if isinstance(event, InputRequestEvent):
            event.content.respond(input(event.prompt))

    context += response.summary

print(response.summary)

# Nested Chat

Nested chat can be implemented using tool calling which initiate a new chat using the run function internally

In [ ]:
@tool()
def plan_lesson(topic: str) -> str:
    with llm_config:
        planner = ConversableAgent(
            name="Planner",
            system_message="You are a planner. Collaborate with reviewer to create a lesson plan.",
        )

        reviewer = ConversableAgent(
            name="Reviewer",
            system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes. Say DONE! when satisfied with the lesson plan.",
        )

    response = run(
        sender=reviewer,
        receiver=planner,
        message=f"Create lesson plans for 4th grade on the topic of: {topic}",
        run_pattern=run_pattern,
        terminate_on=Keyword("DONE!") | MaxTurns(10),
    )

    for event in response.events:
        print(f"Got event: {event.type=}, {event=}")
        if isinstance(event, InputRequestEvent):
            event.content.respond(input(event.prompt))

    return response.summary


@tool()
def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


teacher = ConversableAgent(
    name="Teacher",
    system_message="You are a teacher. Choose topics and create a plan using plan_lesson. Say DONE! when finished.",
)

plan_lesson.register_for_llm(teacher)
submit_plan.register_for_llm(teacher)


response = run(
    teacher,
    message="Create lesson plans for 4th grade.",
    run_pattern=run_pattern,
    terminate_on=Keyword("DONE!") | MaxTurns(10),
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)